In [1]:
from IPython.display import display, HTML
from matplotlib import colors, cm
import matplotlib.pyplot as plt
import networkx as nx
import netgraph
import os
#import json
#current_location = os.getcwd()
#os.chdir('problems/set1')
#os.listdir()
directory = {'problems':'problems/set1', 'definitions':'definitions', 'rules': 'rules'}

## Templates
Pakuriam keletą automatinių HTML skriptų generatorių gražiam dizainui

In [2]:
class CustomHTML:
    '''various operations of template scripts kept line by line'''
    # by https://www.w3.org/community/webed/wiki/HTML/Training/Tag_syntax
    # https://www.thoughtco.com/html-singleton-tags-3468620
    directory = directory['problems']
    tab = 4
    void = ('area', 'base', 'br', 'col', 'command', 'embed', 'hr', 'img', 'input', 
            'keygen', 'link', 'meta', 'param', 'source', 'track', 'wbr')
       
    def __init__(self):
        pass
    
    def tabbing(self, text):
        '''adds indent to each line'''
        return '\n'.join([' '*self.tab + line for line in text.split('\n')])
    
    def element(self, tag_name, content='', **options):
        """if content is string, add start tag and end tag 
        in the same row (unless it's void, no end tag then).
        Otherwise include each item of content in new line using indentation"""
        attr_space = ' '.join(f'{key}="{val}"' for key, val in options.items())
        if attr_space: 
            start_tag = f'<{tag_name} {attr_space}>'
        else: 
            start_tag = f'<{tag_name}>'
            
        if tag_name in self.void:
            return start_tag
        else:
            end_tag = f'</{tag_name}>'
            if type(content) is str:
                return f'{start_tag} {content} {end_tag}'
            else:
                content = "\n".join([self.tabbing(c) for c in content])
                return f'{start_tag}\n{content}\n{end_tag}'
    
    def hjoin(self, items):
        text = '\n&emsp;\n'.join([self.element('div', [item]) for item in items])
        return self.element('div', [text], style='display: flex')
        
    def vjoin(self, items, sep='\n&emsp;\n'):
        return sep.join(items)
    
    def itemize(self, content):
        if type(content) is str:
            raise TypeError('You should itemize bullet list, not str')
        return self.element('ul', [self.element('li', c) for c in content])
    
    def brezinys(self, brezinys):
        return self.element('img', src=os.path.join(self.directory, brezinys), style='height: 200px')
    
    def duota_rasti(self, duota, rasti):
        return self.vjoin([self.element('h4', 'DUOTA:'), self.itemize(duota),
                          self.element('h4', 'RASTI:'), self.itemize(rasti)], sep='\n')
    
    def brezinys_duota_rasti(self, brezinys, duota, rasti, setup=dict()):
        return self.hjoin([self.brezinys(brezinys), self.duota_rasti(duota, rasti)])
        
    def sprendimas(self, steps):
        return self.vjoin([self.element('h3', 'SPRENDIMAS:'), self.itemize(steps)], sep='\n')
    
script = CustomHTML().brezinys_duota_rasti('my_draw.jpg', ['$a$', '$b$'], ['$c$'])
print(script)

<div style="display: flex">
    <div>
        <img src="problems/set1/my_draw.jpg" style="height: 200px">
    </div>
    &emsp;
    <div>
        <h4> DUOTA: </h4>
        <ul>
            <li> $a$ </li>
            <li> $b$ </li>
        </ul>
        <h4> RASTI: </h4>
        <ul>
            <li> $c$ </li>
        </ul>
    </div>
</div>


In [3]:
HTML(script)

## Uždavinio sandara
* Uždavinys - tai žinomųjų rinkinys $a_1$, $a_2$, ..., $a_n$ kartu su nurodymu pateikti sprendimą, kaip rasti vieną ar kelis nežinomuosius $x_1$, $x_2$, ..., $x_n$. Senesnis vaizdavimas uždavinio kaip Toulmino modeliu pagrįstas žemėlapis buvo ne toks lankstus, nes dažnai taisyklės taikomos keliems duomenims ar jų išvadoms, o jas sukomplektuoti dažnai yra sunku. 
  
* Uždaviniui galima:
  * rasti tarpines sprendimo būsenas turint sprendimo žingsnius (klausimas, sprendimas)
  * paskelbti tam tikrą sprendimo žingsnių rinkinį nauju uždaviniu
  * išbraižyti grafą  

Which kinds of data do I expect during solution process? 
* The one that forms `Data.dataspace`: known and unknown one, mentioned in condition of problem.
Known, unknown or derived.
* The one that forms `Data.stepspace`: implicated one.

Note that both abovementioned are dictionaries of indices. Data itself is storied `Data.references` that maps these indices into instances of classes.

In [4]:
class Data:
    dataspace = {'known': [], 'unknown': []}
    stepspace = {'ifthen': []}
    references = dict()
    
    def __init__(self, n, fr=None, to=None, by=None):
        self.n = n
        if fr is not None:
            self.fr = fr
        if to is not None:
            self.to = to
        if by is not None:
            self.by = by
    
    def reset(self):
        self.dataspace = {'known': [], 'unknown': []}
        self.stepspace = {'ifthen': []}
        self.references = dict()
        
        
class Known(Data):
    '''Descriptor for a known data of text problem refered in condition.
    Responsible for a process of introducing notation if it is required to do so.

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a part of condition that describes a single claim
    to: notation introduced (optional)
    by: general environment required to introduce notation; 
        usually a relation that refers to some objects or quantities used
    '''
    
    def __init__(self, *args, **kwargs):
        Data.__init__(self, *args, **kwargs)
        Data.dataspace['known'].append(self.n)
        Data.references[self.n] = self
    
class Unknown(Data):
    '''Descriptor for a data to look for refered in condition.
    Responsible for a process of introducing notation if it is required to do so.

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a part of condition that refers what to look for
    to: notation introduced (optional)
    by: general environment required to introduce notation; 
        usually an object or quantity
    '''
    
    def __init__(self, *args, **kwargs):
        Data.__init__(self, *args, **kwargs)
        Data.dataspace['unknown'].append(self.n)
        Data.references[self.n] = self
        
class IfThen(Data):
    '''Descriptor for a derived data from data found or given already
    Responsible for a process of deriving a new data using rules and definitions

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a minimal set of data needed to derive a current claim
    to: notation introduced (optional)
    by: general environment that is a rationale for deriving a current claim; 
        usually a main rule or definition to apply on a minimal set of data
    ask: a question that includes a minimal set of data and asks to derive a current claim
    background: additional environment that is to weak intuitively to be interpreted
        as a rationale for deriving a new claim but plays an important role in a
        process in derivation
    '''
    
    def __init__(self, *args, carrier=None, ask=None, background=(), **kwargs):
        if carrier is not None:
            self.carrier = carrier
        if ask is not None: 
            self.ask = ask
        self.background = background
        
        Data.__init__(self, *args, **kwargs)
        Data.stepspace['ifthen'].append(self.n)
        Data.references[self.n] = self

Which kinds of reasoning do I expect in each step notation and solution process?

Initially, there is a list of actions possible:
* Use **definition** in implication
* Use **rule** in implication
* Use assignment:
    * assign an equation like `trukmė yra trys minutės` while interpreting condition
    * derive an equation like `vadinasi, pagal ankstesnes sąlygas, atkarpos ilgis yra x+3`

In [5]:
class Warrant:
    def __init__(self, *args, **kwargs):
        self.args=args
        self.kwargs=kwargs

class Derive(Warrant):
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
        
class Assign(Warrant):
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
    
    def __repr__(self):
        return ''
        
class Def(Warrant):
    directory = directory['definitions']
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
        
    def __repr__(self):
        cell = self.args[0].replace(' ','-')
        script = CustomHTML().element('a', f'PAGAL {self.args[0]} apibrėžimą', 
                                      href=os.path.join(self.directory, f'definitions.ipynb#{cell}'),
                                      target="_blank")
        return script
        
class Rule(Warrant):
    directory = directory['rules']
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
        
    def __repr__(self):
        cell = self.args[0].replace(' ','-')
        script = CustomHTML().element('a', f'PAGAL {self.args[0]}', 
                                      href=os.path.join(self.directory, f'rules.ipynb#{cell}'),
                                      target="_blank")
        return script

Can we classify a data that participates in application of rules, definitions, assignments and derivations?

* Quantity that exists as multitude or magnitude (you need to specify a type)
* Variable that exists as unknown, parameter or number (you need to specify a type)

Which processes and parameters are involved in solution of problem?

In [6]:
class Problem:
    def __init__(self, task, data, steps, name=None, brezinys=None, analogies=None, conclusion=None, author=None, picture=None):
        self.task=task
        self.data=data
        self.steps=steps
        self.analogies=analogies
        self.conclusion=conclusion
        self.author=author
        self.picture=picture  
        self.name=name
        self.metadata={'data': Data.dataspace, 'steps': Data.stepspace, 'references': Data.references}
        
        #clearing data after initialisation while keeping attributes in metadata
        Data.dataspace = {'known': [], 'unknown': []}
        Data.stepspace = {'ifthen': []}
        Data.references = dict()

    #@staticmethod
    def explain(self, i):
        construct = pr.metadata['references'][i]
        if hasattr(construct, 'by'):
            explanation = f'{construct.by}'
        else: 
            explanation = ''
            
        if hasattr(construct, 'carrier'): solution = construct.carrier
        elif hasattr(construct, 'to'): solution = construct.to
        else: solution = construct.fr

        return f'{solution}{explanation}'
    
    def conclude(self, i):
        construct = pr.metadata['references'][i]
        if hasattr(construct, 'to'): solution = construct.to
        elif hasattr(construct, 'carrier'): solution = construct.carrier
        else: solution = construct.fr
        return solution      
    
    def return_substeps(self, return_solution=False, no_brezinys=False):        
        views = []
        implications = self.metadata['steps']['ifthen']
        references = self.metadata['references']
        for step_id, i in enumerate(implications): 
            step = references[i]
            duota = tuple(self.explain(index) for index in step.fr)
            rasti = [step.ask]
            header1, pradinis_brezinys, brezinys, sprendimas = '', '', '', ''
            
            if (self.picture is not None) or not(no_brezinys):
                header1 = CustomHTML().element('h3', f'PRADINIS BRĖŽINYS')
                pradinis_brezinys = CustomHTML().brezinys(self.picture)
                
            if hasattr(step, 'picture'):
                picture = step.picture
            else:
                picture = self.picture
                
            if return_solution:
                sprendimas = CustomHTML().sprendimas([self.explain(i)])
         
            header2 = CustomHTML().element('h3', f'SĄLYGA: žingsnis {step_id+1}')
            view = CustomHTML().brezinys_duota_rasti(picture, duota, rasti)            
            v = CustomHTML().vjoin([header1, pradinis_brezinys, header2, view, sprendimas])
            views.append(v)
        return views
            
    def return_steps(self, return_solution=False, no_brezinys=False):   
        sprendimas, pradinis_brezinys, view = '', '', ''
        title = CustomHTML().element('h2', f'{self.name}')
        sąlyga = self.task

        if (self.picture is not None) or not(no_brezinys):
            pradinis_brezinys = pr.picture
            duota = [self.explain(i) for i in self.metadata['data']['known']]
            rasti = [self.explain(i) for i in self.metadata['data']['unknown']]
            view = CustomHTML().brezinys_duota_rasti(pradinis_brezinys, duota, rasti)

        if return_solution:
            steps = [self.explain(i) for i in self.metadata['steps']['ifthen']]
            sprendimas = CustomHTML().sprendimas(steps)
            
        return CustomHTML().vjoin([title+sąlyga, view, sprendimas], sep='<br><br>\n')

Pakurkim klasę uždavinio sprendimo procesui aprašyti

In [7]:
pr = Problem(
task = 'Sujungus keturkampio kraštinių vidurio taškus gautas lygiagretainis. Raskite jo perimetrą, jei viena jo įstrižainė lygi $x$, o kita $y$',
picture = 'my_draw.jpg',
data=(
    Known(n=0, fr='$X$ - atkarpos $AB$ vidurio taškas'),
    Known(n=1, fr='$Y$ - atkarpos $BC$ vidurio taškas'),
    Known(n=2, fr='$Z$ - atkarpos $CD$ vidurio taškas'),
    Known(n=3, fr='$T$ - atkarpos $DA$ vidurio taškas'),
    Known(n=4, fr='Viena lygiagretainio įstrižainė lygi $x$', to='$AC=x$', by=Assign('įstrižainė', '$x$')),
    Known(n=5, fr='Kita lygiagretainio įstrižainė lygi $y$',  to='$BD=y$', by=Assign('įstrižainė', '$y$')),
    Unknown(n=6, fr='Raskite lygiagretainio perimetrą', to='$P_{XYZT}$')),  
steps=(
    IfThen(n=7, fr=(0, 1), carrier=r'$XY$ yra $\triangle ABC$ vidurio linija', 
           by=Def('Vidurio linija'), ask='Kokia savybe pasižymi atkarpa $XY$?'),
       
    IfThen(n=8, fr=(2, 3), carrier=r'$ZT$ yra $\triangle ACD$ vidurio linija', 
           by=Def('Vidurio linija'), ask='Kokia savybe pasižymi atkarpa $ZT$?'),
    
    IfThen(n=9, fr=(1, 2), carrier=r'$YZ$ yra $\triangle ABD$ vidurio linija', 
           by=Def('Vidurio linija'), ask='Kokia savybe pasižymi atkarpa $YZ$?'),
    
    IfThen(n=10, fr=(0, 3), carrier=r'$XT$ yra $\triangle BCD$ vidurio linija', 
           by=Def('Vidurio linija'), ask='Kokia savybe pasižymi atkarpa $XT$?'),
    
    IfThen(n=11, fr=(7,4), carrier=r'$\displaystyle XY = \frac{x}{2}$', 
           by=Rule('Vidurio linijos savybė'), ask='$XY$', background=(Rule('deassign'), 4)),
    
    IfThen(n=12, fr=(8,4), carrier=r'$\displaystyle ZT = \frac{x}{2}$', 
           by=Rule('Vidurio linijos savybė'), ask='$ZT$', background=(Rule('deassign'), 4)),
    
    IfThen(n=13, fr=(9,5), carrier=r'$\displaystyle YZ = \frac{y}{2}$', 
           by=Rule('Vidurio linijos savybė'), ask='$YZ$', background=(Rule('deassign'), 5)),
    
    IfThen(n=14, fr=(10,5), carrier=r'$\displaystyle XT = \frac{y}{2}$', 
           by=Rule('Vidurio linijos savybė'), ask='$XT$', background=(Rule('deassign'), 5)),
    
    IfThen(n=15, fr=(11,12,13,14), 
           carrier=r'$P_{XYZT}=XY+ZT+XY+ZT=\frac{x}{2}+\frac{x}{2}+\frac{y}{2}+\frac{y}{2}=x+y$',
           to=r'$P_{XYZT}=x+y$',
           by=Def('Perimetras'), 
           ask='$P_{XYZT}$',
           background=(Rule('trupmenų su vienodais vardikliais sudėtis'), Rule('deassign'), 11,12,13,14))),
    
analogies = [(0,1,2,3), (4,5), (7,8,9,10), (11,12,13,14)],
conclusion = 'Lygiagretainio, gauto sujungus keturkampio kraštinių vidurio taškus, perimetras lygus to keturkampio įstrižainių sumai',
author = 'vadovėlis gimnazijų 1 ir 2 klasėms',
name = 'Uždavinys 99')

Svarbiausia informacija automatiškai susidėlioja į klasės `Data` kintamuosius vos tik yra sukuriami `IfThen`, `Known`, `Unknown` kintamieji. Kiekvienąsyk kuriant naują `Problem` egzempliorių ši informacija nusiresetina, o paskui pasipildo reikiamais duomenimis automatiškai.

In [8]:
pr.metadata

{'data': {'known': [0, 1, 2, 3, 4, 5], 'unknown': [6]},
 'steps': {'ifthen': [7, 8, 9, 10, 11, 12, 13, 14, 15]},
 'references': {0: <__main__.Known at 0x7f640dcb9ca0>,
  1: <__main__.Known at 0x7f640dcb97f0>,
  2: <__main__.Known at 0x7f640dcb9460>,
  3: <__main__.Known at 0x7f640dcb9760>,
  4: <__main__.Known at 0x7f640dcb93d0>,
  5: <__main__.Known at 0x7f640dcb9520>,
  6: <__main__.Unknown at 0x7f640dcb94c0>,
  7: <__main__.IfThen at 0x7f640dcd8a60>,
  8: <__main__.IfThen at 0x7f640dcd8760>,
  9: <__main__.IfThen at 0x7f640dcd8eb0>,
  10: <__main__.IfThen at 0x7f640dcd8df0>,
  11: <__main__.IfThen at 0x7f640dcd8be0>,
  12: <__main__.IfThen at 0x7f640dcd8fd0>,
  13: <__main__.IfThen at 0x7f640dcbb970>,
  14: <__main__.IfThen at 0x7f640dcbb820>,
  15: <__main__.IfThen at 0x7f640dcbb640>}}

## Realizacija
Kiekvienam sprendimo žigsniui pasižiūri, iš kokių duomenų id jis gautas, o tada nukreipia į jų tekstą (jau sutvarkytą maksimaliai, nepriklausomai nuo to, ar buvo atliktas žymėjimas, ar išvedimas)

Prielaidos: duomenys gali turėti tik tipus `known` ir `unknown`. Sprendimo žingsniai kol kas yra tik formos `ifthen`, tačiau gal ateityje prisidės `case`, `contradiction`, `induction` ir pan.

In [9]:
subtasks = pr.return_substeps(return_solution=True)
HTML(subtasks[8])

In [10]:
h = pr.return_steps(return_solution=True, no_brezinys=False)
HTML(h)

## Experimental plugin to problem class

In [15]:
%matplotlib qt5

def draw_graph(pr):
    fig, ax = plt.subplots(figsize=(5, 5))
    G=nx.DiGraph()
    G.add_nodes_from(pr.metadata['data']['known'], color = 'green')
    for i in pr.metadata['steps']['ifthen']:
        for node in pr.metadata['references'][i].fr:
            G.add_node(i, color='lightblue')
            G.add_edge(node, i, by=pr.metadata['references'][i].by.args[0])


    node_vals = dict()
    tg = list(nx.topological_generations(G))
    norm = colors.Normalize(vmin=0, vmax=len(tg)-1)
    for i, generation in enumerate(tg):
        for node in sorted(generation):
            color = tuple(n/255 for n in cm.summer(norm(len(tg)-1-i), bytes=True))
            node_vals[node] = color

    basespace = nx.get_edge_attributes(G, 'by').values()
    lookin = list(set(basespace))
    idx = [lookin.index(n) for n in basespace]
    norm = colors.Normalize(vmin=0, vmax=max(idx))
    edge_colors = [tuple(n/255 for n in cm.rainbow(norm(v), bytes=True)) for v in idx]


    annotations = {idx:pr.conclude(idx) for idx in pr.metadata['references'] 
                   if pr.metadata['references'][idx].__class__.__name__ != 'Unknown'}

    I = netgraph.InteractiveGraph(G, node_size=6, 
                                  node_labels=True, 
                                  node_color=node_vals,
                                  edge_color = dict(zip(nx.get_edge_attributes(G, 'by').keys(), edge_colors)),
                                  annotations=annotations,
                                  edge_width=2, arrows=True, ax=ax)

    display(HTML('<h2> Data <h2>'))
    for key, val in annotations.items():
        display(HTML(f'{key}: {val}'))

    display(HTML('<h2> Rules <h2>'))
    for key, val in zip(nx.get_edge_attributes(G, 'by').keys(), basespace):
        display(HTML(f'{key}: {val}'))
        
    return I

draw_graph(pr)

In [ ]:
import matplotlib.pyplot as plt